In [1]:
!pip uninstall torch -y
!pip uninstall torch-scatter -y
!pip uninstall torch-sparsse -y
!pip uninstall torchvision -y
!pip uninstall torchaudio -y
!pip uninstall torch-spline-con -y
!pip uninstall torch-cluster -y
!pip uninstall torch-geometric -y

In [2]:
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio===0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached https://download.pytorch.org/whl/cu111/torch-1.9.0%2Bcu111-cp38-cp38-win_amd64.whl (3128.1 MB)
  Using cached https://download.pytorch.org/whl/cu111/torchvision-0.10.0%2Bcu111-cp38-cp38-win_amd64.whl (2.5 MB)
  Using cached torchaudio-0.9.0-cp38-cp38-win_amd64.whl (215 kB)


In [3]:
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.9.0+cu111.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.9.0+cu111.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.9.0+cu111.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.9.0+cu111.html
!pip install torch-geometric

Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu111.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu111.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu111.html
  Using cached https://pytorch-geometric.com/whl/torch-1.9.0%2Bcu111/torch_cluster-1.5.9-cp38-cp38-win_amd64.whl (1.2 MB)
Looking in links: https://pytorch-geometric.com/whl/torch-1.9.0+cu111.html
  Using cached https://pytorch-geometric.com/whl/torch-1.9.0%2Bcu111/torch_spline_conv-1.2.1-cp38-cp38-win_amd64.whl (445 kB)
Processing c:\users\emrek\appdata\local\pip\cache\wheels\1a\f4\19\2332058c82a978e7ef746a160eb1d7c181c7ba25f16cd01333\torch_geometric-1.7.2-py3-none-any.whl
  Using cached googledrivedownloader-0.4-py2.py3-none-any.whl (3.9 kB)
Processing c:\users\emrek\appdata\local\pip\cache\wheels\c8\68\21\d04741956802581119d875a0d9fce6895e8f9ffae71223f3e3\python_louvain-0.15-py3-none-any.whl
  Using cached isodate-0.6.0-py2.py3-none-any.whl (45 kB)


In [4]:
import torch
print(torch.__version__)
print(torch.version.cuda)

1.9.0+cu111
11.1


In [2]:
from typing import Union, Tuple
from torch_geometric.typing import OptPairTensor, Adj, Size # Optional[Tensor], Union[Tensor, SparseTensor], Optional[Tuple[int, int]], all about data type

from torch import Tensor
from torch.nn import Linear
import torch.nn.functional as F
from torch_sparse import SparseTensor, matmul
from torch_geometric.nn.conv import MessagePassing

In [3]:
import warnings
warnings.filterwarnings("ignore")
import os
import time
import random
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from joblib import Parallel, delayed

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import torch
import torch.nn.functional as F
import torch_geometric.nn as gnn
from torch_geometric.nn import MessagePassing
from torch_geometric.data import Dataset, Data, DataLoader
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from torch_geometric.utils import softmax

In [4]:
df = pd.read_csv('input/CMS_trigger.csv')

print(df.columns)
df.head()

Index(['Unnamed: 0', 'Phi_0', 'Phi_2', 'Phi_3', 'Phi_4', 'Theta_0', 'Theta_2',
       'Theta_3', 'Theta_4', 'BendingAngle_0', 'BendingAngle_2',
       'BendingAngle_3', 'BendingAngle_4', 'TimeInfo_0', 'TimeInfo_2',
       'TimeInfo_3', 'TimeInfo_4', 'RingNumber_0', 'RingNumber_2',
       'RingNumber_3', 'RingNumber_4', 'Front_0', 'Front_2', 'Front_3',
       'Front_4', 'Mask_0', 'Mask_2', 'Mask_3', 'Mask_4',
       'PatternStraightness', 'Zone', 'MedianTheta', 'q/pt', 'PhiAngle',
       'EtaAngle'],
      dtype='object')


,Unnamed: 0,Phi_0,Phi_2,Phi_3,Phi_4,Theta_0,Theta_2,Theta_3,Theta_4,BendingAngle_0,...,Mask_0,Mask_2,Mask_3,Mask_4,PatternStraightness,Zone,MedianTheta,q/pt,PhiAngle,EtaAngle
0,0,58.066666,55.466667,55.466667,56.000000,11.970,11.684999,11.400,11.400,-13.000000,...,0.0,0.0,0.0,0.0,5.0,3.0,105.0,0.181209,-3.075936,1.722345
1,1,64.583336,66.800000,67.066666,67.200000,6.555,6.840000,6.555,6.840,7.000000,...,0.0,0.0,0.0,0.0,3.0,1.0,124.0,-0.146131,-0.167139,2.012122
2,2,71.150000,67.033330,66.266670,65.466670,2.850,2.565000,2.280,2.280,-16.613783,...,0.0,0.0,0.0,0.0,7.0,0.0,127.0,0.367024,2.431823,2.321646
3,3,34.933334,31.200000,31.200000,31.833334,9.690,8.835000,8.835,9.120,-13.000000,...,0.0,0.0,0.0,0.0,6.0,2.0,60.0,0.267774,-1.343305,1.854506
4,4,68.150000,68.266670,68.300000,68.400000,2.565,2.565000,2.565,2.565,0.000000,...,0.0,0.0,0.0,0.0,4.0,0.0,128.0,-0.019179,-3.134433,2.333772


In [5]:
pT = abs(1/df.loc[:,'q/pt'])
inv_pT = 1/pT
pT.shape

(1179356,)

In [6]:
features = ['Phi_'+str(i) for i in [0,2,3,4]] + ['Theta_'+str(i) for i in [0,2,3,4]] + ['Front_'+str(i) for i in [0,2,3,4]]

scaler_1 = StandardScaler()
df[features] = scaler_1.fit_transform(df[features])

In [7]:
edge_index = torch.tensor([(0,1),(1,2),(2,3),(3,2),(2,1),(1,0)], dtype=torch.long).T
X_data = df[features].to_numpy()
Y_data = pT.copy()
# Y_data_invpT = inv_pT.copy()
print(X_data.shape)
print(Y_data.shape)

(1179356, 12)
(1179356,)


In [8]:
def process_data(i):
  graph = X_data[i].reshape(-1,4).T
  y = Y_data[i]
  data = Data(x=torch.tensor(graph, dtype=torch.float), y=torch.tensor(y, dtype=torch.float), edge_index=edge_index)
  return data